In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
res_dir <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp_nc'

phewas_hits_ld <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.tsv')
phewas_hits_ld_gwas_catalog_summary <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.gwascatalog.summary.tsv')

phewas_hits_ld_finngen <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.finngen.tsv')


In [3]:
finngen_R2_extracted <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.gwascatalog.novel.finngen.R2.tsv') %>%
fread(sep='\t') %>%
rename('chrom' = '#chrom')


In [4]:
finngen_R2_manifest <- '/scratch/groups/mrivas/users/ytanigaw/20200114_FinnGen_R2/summary_stats/r2_manifest.tsv' %>%
fread(sep='\t')


In [19]:
finngen_R2_df <- finngen_R2_extracted %>% 
mutate(var_id = paste(chrom, pos, ref, alt, sep='-')) %>%
left_join(
    finngen_R2_manifest  %>%
    select(phenocode, name), 
    by='phenocode'
) %>%
select(-chrom, -pos, -ref, -alt)

finngen_R2_df.colnames <- colnames(finngen_R2_df) %>%
lapply(function(x){paste0('finngen_', x)})

colnames(finngen_R2_df) <- finngen_R2_df.colnames

In [21]:
df <- fread(phewas_hits_ld) %>% rename('CHROM' = '#CHROM')


In [22]:
gwas_c_summary <- fread(phewas_hits_ld_gwas_catalog_summary) %>%
rename('CHROM' = '#CHROM')


In [25]:
idx_df <- df %>%
arrange(CHROM, POS, GBE_ID) %>%
select(ID) %>%
unique()%>%
mutate(phewas_idx = 1:n())


In [26]:
merged_df <- df %>% 
left_join(
    idx_df, by=c('ID')
) %>%
left_join(
    gwas_c_summary  %>% select(ID, GBE_ID, is_novel),
    by=c('ID', 'GBE_ID')
) %>% 
full_join(
    finngen_R2_df, 
    by='finngen_var_id'
) %>%
drop_na(LD_ID) 


In [30]:
merged_df %>% 
rename('#CHROM' = 'CHROM') %>%
fwrite(phewas_hits_ld_finngen, sep='\t')


In [31]:
phewas_hits_ld_finngen

[1] "/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp_nc/ukb24983_imp_v3.nc.phewas.hits.ld.finngen.tsv"